In [ ]:
import pandas as pd
import pickle as p
import os
data_path = './data/'
log_path = './log/'
if not os.path.exists(log_path):
    os.mkdir(log_path)
csv = pd.read_csv(data_path+'train_set.csv')

In [6]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, Dense, Dropout, Activation
from keras.layers.recurrent import LSTM

def get_simple_cnn(vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
        embeddings_initializer='random_normal',
            trainable=True
        ))
    model.add(Conv1D(activation="tanh",
                         filters=64,
                     kernel_size=1,
                     padding="valid"))
    model.add(Conv1D(activation="tanh",
                         filters=128,
                     kernel_size=2,
                     padding="valid"))
    model.add(Conv1D(activation="tanh",
                         filters=256,
                     kernel_size=3,
                     padding="valid"))
    model.add(Conv1D(activation="tanh",
                         filters=512,
                     kernel_size=3,
                     padding="valid"))
    model.add(Dense(2048, activation='sigmoid'))
    model.add(Dense(512,  activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(19))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
m = get_simple_cnn(70000, 256)

In [9]:
m.layers

In [12]:
def get_RNN(vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
        embeddings_initializer='random_normal',
            trainable=True
        ))
    model.add(LSTM(256,return_sequences=True))
    model.add(LSTM(256,dropout=0.5,return_sequences=False))
    model.add(Dense(19))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
m = get_RNN(70000, 256)

In [15]:
from keras.engine import Input
from keras import backend as K
def mix_cnn_rnn(vocab_size, embedding_size):
    input_text = Input(shape=(text_length,), dtype='int32')
    embedding_vec = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
        embeddings_initializer='random_normal',
            trainable=True
        )(input_text)
    cnn_config=[{'kernel_size':1,'filters':64,  padding:'same'},
                {'kernel_size':2,'filters':128,  padding:'same'},
                {'kernel_size':3,'filters':512,  padding:'same'},
                {'kernel_size':2,'filters':128,  padding:'same'}]
    data_aug = []
    for i, c_conf in enumerate(cnn_config):
        if i == 0:
            data_aug.append(Conv1D(c_conf, name='aug_{}st'.format(i+1))(embedding_vec))
            continue
        data_aug.append(Conv1D(c_conf, name='aug_{}st'.format(i+1))(data_aug[i-1]))
    concat_data = concatenate(data_aug)
    rnn_result = LSTM(256,return_sequences=True)(concat_data)
    rnn_result = LSTM(256,dropout=0.5,return_sequences=False)(rnn_result)
    logist = Dense(19, activation='softmax')
    model = Model(input=input_text, output=logist)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
m = get_RNN(70000, 256)